## Imports and libs

In [ ]:
import os
import json
import nltk
import pprint
import numpy as np
import pandas as pd
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk import ngrams
from nltk.stem import RSLPStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import string
import unicodedata
import re
from wordcloud import WordCloud
from pandasql import sqldf
import spacy

#python -m spacy download pt 
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('rslp')
# pip install pandasql

## Utils and configs

In [ ]:
# Configs

reviewsVolume = 20000
maxBowSize = 2000
commonStopWords = {'pra', 'ca', 'so', 'ja', ''}
export_folder = 'results_data'

In [ ]:
# Utils

def getJsonFileList():
    found_files = []
    directory = 'outputs'
     
    for filename in os.listdir(directory):
        if "reviews-" in filename and "-total" in filename:
            found_files.append(filename)
    
    return found_files

def getPersistedReviews():
    file_import = f"outputs/reviews-multi-company-merged-{reviewsVolume}.json"

    with open(file_import, 'r', encoding='utf-8') as arquivo:
        data = json.load(arquivo)
        return data

def getPersistedJson(jsonPath):
    with open(jsonPath, 'r', encoding='utf-8') as arquivo:
        data = json.load(arquivo)
        return data

# Remover vírgulas e pontos dos textos
def removePunctuationUnique(string):
    string = ''.join(c for c in unicodedata.normalize('NFD', string) if unicodedata.category(c) != 'Mn')
    string = re.sub(r'[^\w\s]', '', string)
    return string

def removePunctuation(texts):
    return [removePunctuationUnique(text) for text in texts]

# Tokenizar os textos em palavras
def tokenize(texts):
    return [word.lower() for text in texts for word in word_tokenize(text, language='portuguese')]
    
# Remover stopwords
def removeStopwords(tokens):
    stopWords = set(stopwords.words('portuguese')) 
    wordsToRemove = stopWords.union(commonStopWords)
    return [word for word in tokens if word not in wordsToRemove]

def removeStopwordsList(frases):
    stopWords = set(stopwords.words('portuguese')) 
    wordsToRemove = stopWords.union(commonStopWords)
    frasesSemStopwords = []
    for frase in frases:
        tokens = frase.split()  # Dividir a frase em tokens
        tokensSemStopwords = [word for word in tokens if word not in wordsToRemove]
        novaFrase = ' '.join(tokensSemStopwords)  # Juntar os tokens novamente em uma frase
        frasesSemStopwords.append(novaFrase)
    return frasesSemStopwords

def dataCleaning(texts):
   
    return tokens

# Aplicar stemming nas palavras
def applyStemming(tokens):
    stemmer = RSLPStemmer()
    return [stemmer.stem(word) for word in tokens]

# Criar pares consecutivos de tokens
def createPairs(words, pairNumber = 2):
    less = pairNumber - 1
    return [' '.join(words[i:i+pairNumber]) for i in range(len(words)-less)]

def generateWordCloud(list, output):
    # Criar dicionário de palavras e frequências
    palavras_frequencias = {}
    for palavra, frequencia in list:
        palavras_frequencias[palavra] = frequencia

    wordcloud = WordCloud().generate_from_frequencies(palavras_frequencias)
    wordcloud.to_file(output)

def create_heatmap(similarity, cmap = "YlGnBu"):
  df = pd.DataFrame(similarity)
  df.columns = labels
  df.index = labels
  fig, ax = plt.subplots(figsize=(5,5))
  sns.heatmap(df, cmap=cmap)

def generateHorizontalLineGraph(graphdf, output, title, xlabel, ylabel, sortby, kind='barh', width=10, height=6, ascending=True):
    by = sortby if sortby else xlabel

    #if by != 'NOOP':
    graphdf = graphdf.sort_values(by=by, ascending=ascending)

    graphdf = graphdf.head(15)
    
    graph = graphdf.plot(x=ylabel, y=xlabel, kind=kind, figsize=(width, height))
    #plt.figure(figsize=(10, 6))
    graph.invert_yaxis()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.savefig(output)
    #plt.tight_layout()
    plt.close()

    #return graph

def getLemmatization(texts):
    nlp = spacy.load('pt_core_news_sm', disable=['ner', 'parser'])
    total = len(texts)

    print(f"Starting lemmatization from: {total} of texts")

    listpalavras = []
    docs = []
    count = 0

    for text in texts:
        current = count + 1
        tokens = tokenize([text])
        doc = nlp(str([palavra for palavra in tokens]))

        lemma = [token.lemma_ for token in doc if token.pos_ != 'PUNCT']

        newText = ' '.join(lemma)
        listpalavras.append(newText)
        docs.append(doc)

        print(f"{current} of {total} texts lemmatized")
        count += 1

    return listpalavras, docs

def saveStoreJson(fileContent, company, store, fileName):
    fileNameBase = f"{export_folder}/reviews-{company}-{store}-{reviewsVolume}.json"
    file_export = fileName if fileName else fileNameBase

    with open(file_export, 'w', encoding='utf-8') as arquivo:
        #arquivo.write(json)
        json.dump(fileContent, arquivo, indent=4, ensure_ascii=False)

def checkIfFileExists(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as arquivo:
            return True
    except:
        return False

# Pre work # 1 - Reading reviews JSON

In [ ]:
# Lendo e filtrando por 2023 

jsonData = getPersistedReviews()
dfJson = pd.DataFrame(jsonData)
dfJson['at'] = pd.to_datetime(dfJson['at'])
dfFilteredData = dfJson[dfJson['at'].dt.year == 2023]
filteredJsonData = json.loads(dfFilteredData.to_json(orient='records'))

## Pre work # 1.1 -  Lemmatização
###### Entrada: Lista de reviews
###### Saída: Lista de reviews com o content lemmatizado

In [ ]:
lemmPathFile = f"results/2023-total-volume-{reviewsVolume}-lemmatizated.json"
lemmatizatedDf = pd.DataFrame()
lemmExists = checkIfFileExists(lemmPathFile)

if lemmExists:
    print("Loading lemmatizated from a existing file")
    lemmatizatedDf = pd.read_json(lemmPathFile)
else:
    print("Generating new lemmatizated file")
    lemmatizatedDf = dfFilteredData
    totalData = lemmatizatedDf.shape[0]
    contentList = lemmatizatedDf['content'].tolist()
    lemmatizatedContent, _ = getLemmatization(contentList)
    lemmatizatedDf["content"] = lemmatizatedContent

    print("Exporting new lemmatizated file")
    lemmatizatedDf.to_json(f"results/2023-total-volume-{reviewsVolume}-lemmatizated.json", orient='records')
    lemmatizatedDf.to_csv(f"results/2023-total-volume-{reviewsVolume}-lemmatizated.csv", index=False)
    lemmatizatedDf.to_excel(f"results/2023-total-volume-{reviewsVolume}-lemmatizated.xlsx", index=False)

lemmatizatedData = json.loads(lemmatizatedDf.to_json(orient='records'))

## Pre work # 1.2 - Creating company formats

In [ ]:
# Lendo o arquivo JSON

reviewsListPTBRApple = []
reviewsListNubankApple = []
reviewsListBBApple = []
reviewsListItauApple = []
reviewsListPTBRGoogle = []
reviewsListNubankGoogle = []
reviewsListBBGoogle = []
reviewsListItauGoogle = []
reviewsListNubank = []
reviewsListItau = []
reviewsListBB = []

#for review in lemmatizatedData:
for review in filteredJsonData:

    content = review['content']

    if review['company'] == 'nubank':
        reviewsListNubank.append(content)

    if review['company'] == 'nubank' and review['store'] == 'Apple':
        reviewsListNubankApple.append(content)
    
    if review['company'] == 'bb':
        reviewsListBB.append(content)

    if review['company'] == 'bb' and review['store'] == 'Apple':
        reviewsListBBApple.append(content)
    
    if review['company'] == 'itau' and review['store'] == 'Apple':
        reviewsListItauApple.append(content)

    if review['company'] == 'itau':
        reviewsListItau.append(content)

    if review['company'] == 'nubank' and review['store'] == 'Google':
        reviewsListNubankGoogle.append(content)
    
    if review['company'] == 'bb' and review['store'] == 'Google':
        reviewsListBBGoogle.append(content)
    
    if review['company'] == 'itau' and review['store'] == 'Google':
        reviewsListItauGoogle.append(content)

    if review['store'] == 'Apple':
        reviewsListPTBRApple.append(content)

    if review['store'] == 'Google':
        reviewsListPTBRGoogle.append(content)

In [ ]:
# Creating payload

bowPayload = [
    {
        'company': 'itau',
        'reviews': reviewsListItau
    },
    {
        'company': 'nubank',
        'reviews': reviewsListNubank
    },
    {
        'company': 'bb',
        'reviews': reviewsListBB
    },
    {
        'company': 'nubank-apple',
        'reviews': reviewsListNubankApple
    },
    {
        'company': 'bb-apple',
        'reviews': reviewsListBBApple
    },
    {
        'company': 'itau-apple',
        'reviews': reviewsListItauApple
    },
    {
        'company': 'all-apple',
        'reviews': reviewsListPTBRApple
    },
    {
        'company': 'nubank-google',
        'reviews': reviewsListNubankGoogle
    },
    {
        'company': 'bb-google',
        'reviews': reviewsListBBGoogle
    },
    {
        'company': 'itau-google',
        'reviews': reviewsListItauGoogle
    },
    {
        'company': 'all-google',
        'reviews': reviewsListPTBRGoogle
    }
]

In [ ]:
# Creating payload

bowPayloadDf = [
    {
        'company': 'itau',
        'reviews': dfFilteredData.loc[dfFilteredData['company'] == 'itau']
    },
    {
        'company': 'nubank',
        'reviews': dfFilteredData.loc[dfFilteredData['company'] == 'nubank']
    },
    {
        'company': 'bb',
        'reviews': dfFilteredData.loc[dfFilteredData['company'] == 'bb']
    },
    {
        'company': 'nubank-apple',
        'reviews': dfFilteredData.loc[(dfFilteredData['company'] == 'nubank') & (dfFilteredData['store'] == 'Apple')]
    },
    {
        'company': 'bb-apple',
        'reviews': dfFilteredData.loc[(dfFilteredData['company'] == 'bb') & (dfFilteredData['store'] == 'Apple')]
    },
    {
        'company': 'itau-apple',
        'reviews': dfFilteredData.loc[(dfFilteredData['company'] == 'itau') & (dfFilteredData['store'] == 'Apple')]
    },
    {
        'company': 'all-apple',
        'reviews': dfFilteredData.loc[dfFilteredData['store'] == 'Apple']
    },
    {
        'company': 'nubank-google',
        'reviews': dfFilteredData.loc[(dfFilteredData['company'] == 'nubank') & (dfFilteredData['store'] == 'Google')]
    },
    {
        'company': 'bb-google',
        'reviews': dfFilteredData.loc[(dfFilteredData['company'] == 'bb') & (dfFilteredData['store'] == 'Google')]
    },
    {
        'company': 'itau-google',
        'reviews': dfFilteredData.loc[(dfFilteredData['company'] == 'itau') & (dfFilteredData['store'] == 'Google')]
    },
    {
        'company': 'all-google',
        'reviews': dfFilteredData.loc[dfFilteredData['store'] == 'Google']
    }
]

# Output # 1 - Lista de comentários por bancos
###### Entrada: Lista (JSON) `reviews-multi-company-merged.json` 
###### Saída: Arquivo XLSX com todos os reviews filtrados por 2023.

In [ ]:
# Create a pandas dataframe
#df = pd.DataFrame(json_data)
#df = df[['company', 'store', 'content']]
# Export the dataframe to xlsx
dfFilteredData.to_excel(f"results/2023-total-volume-{reviewsVolume}.xlsx", index=False)

## Output # 1.2 - Lista de comentários por bancos filtradas
###### Entrada: Lista (JSON) `reviews-multi-company-merged.json` 
###### Saída: Arquivo XLSX com todos os reviews filtrados por 2023 e stop words.

In [ ]:
## Output # 1.2 - 

# Output # 2 -  BoW
###### Entrada: Lista de reviews por empresa: `reviewsListPTBR`, `reviewsListNubank`, `reviewsListBB`, `reviewsListItau`
###### Saída: Lista de BoW de 1, 2, 3, 4 e 5 index por empresa

In [ ]:
listOfFreqDist = []
listOfIndex = [1,2,3,4,5]

def getListOfFrequency(texts, index):
    texts = removePunctuation(texts)
    words = tokenize(texts)
    words = removeStopwords(words)
    #words = applyStemming(words)
    pairs = createPairs(words, index)
    # Calcular a distribuição de frequência das palavras
    freq_dist = FreqDist(pairs)
    most_common_words = freq_dist.most_common()
    return most_common_words

def iterateListOfIndex(texts, company):
    acc = []
    for index in listOfIndex:
        listResult = getListOfFrequency(texts, index)
        acc.append({ 'index': index, 'list': listResult, 'company': company })

    return acc

for item in bowPayload:
    texts = item['reviews'] 
    company = item['company']
    frequencies = iterateListOfIndex(texts, company)
    listOfFreqDist = listOfFreqDist + frequencies
    #df_freq_dist = pd.DataFrame(frequencies, columns=['Word', 'Frequency'])
    #df_freq_dist

In [ ]:
for item in listOfFreqDist:
  company = item['company']
  index = item['index']
  itemList = item['list']

  #pprint(f"Getting {company} index {index} results")
  df = pd.DataFrame(itemList, columns=['Word', 'Frequency'])
  df = df.drop(df.index[maxBowSize:])
  
  print(f"Creating {company} bow index {index} excel file")
  df.to_excel(f"results/{company}/bow-result-index-{index}.xlsx", index=False)

  print(f"Creating {company} bow wordcloud index {index}")
  generateWordCloud(itemList, f"results/{company}/wordcloud-index-{index}.png")

  print(f"Creating {company} index {index} graph file")
  generateHorizontalLineGraph(
    df, 
    f"results/{company}/bow-graph-index-{index}.png", 
    f"BoW {company} index {index}", 
    "Frequency", 
    "Word",
    width = 10 + (index * 4),
    height = 6, 
    sortby = "Frequency",
    ascending = False
  )

# Output # 2.1 -  BoW por Score
###### Entrada: Lista de reviews
###### Saída: Lista de BoW de 1, 2, 3, 4 e 5 index por score por empresa

In [ ]:
listOfIndex = [1,2,3,4,5]

def getListOfFrequency(texts, index):
    texts = removePunctuation(texts)
    words = tokenize(texts)
    words = removeStopwords(words)
    #words = applyStemming(words)
    pairs = createPairs(words, index)
    # Calcular a distribuição de frequência das palavras
    freq_dist = FreqDist(pairs)
    most_common_words = freq_dist.most_common()
    return most_common_words

def generateInfos(texts, company, index, score):
   #pprint(f"Getting {company} index {index} results")
    dfinfo = pd.DataFrame(texts, columns=['Word', 'Frequency'])
    dfinfo = dfinfo.drop(df.index[maxBowSize:])
    
    print(f"Creating {company} bow index {index} excel file for score {score}")
    dfinfo.to_excel(f"results/{company}/score/{score}/score-bow-result-index-{index}.xlsx", index=False)

    print(f"Creating {company} bow wordcloud index {index} for score {score}")
    generateWordCloud(texts, f"results/{company}/score/{score}/score-wordcloud-index-{index}.png")

    print(f"Creating {company} index {index} graph file for score {score}")
    generateHorizontalLineGraph(
        dfinfo, 
        f"results/{company}/score/{score}/score-bow-graph-index-{index}.png", 
        f"BoW {company} index {index}", 
        "Frequency", 
        "Word",
        width = 10 + (index * 4),
        height = 6, 
        sortby = "Frequency",
        ascending = False
    )

def iterateListOfIndex(texts, company, score):
    for index in listOfIndex:
        listResult = getListOfFrequency(texts, index)
        generateInfos(listResult, company, index, score)

def iterateScores(item):
    reviews = item['reviews']
    company = item['company']
    reviews = pd.DataFrame(reviews)

    scoreList = {
        '1': reviews.loc[reviews['score'] == 1],
        '2': reviews.loc[reviews['score'] == 2],
        '3': reviews.loc[reviews['score'] == 3],
        '4': reviews.loc[reviews['score'] == 4],
        '5': reviews.loc[reviews['score'] == 5]
    }

    for score, scoreItem in scoreList.items():
        texts = scoreItem['content'].tolist()
        iterateListOfIndex(texts, company, score)

for item in bowPayloadDf:
    iterateScores(item)

In [ ]:
for item in listOfFreqDist:
  company = item['company']
  index = item['index']
  itemList = item['list']

  #pprint(f"Getting {company} index {index} results")
  df = pd.DataFrame(itemList, columns=['Word', 'Frequency'])
  df = df.drop(df.index[maxBowSize:])
  
  print(f"Creating {company} bow index {index} excel file")
  df.to_excel(f"results/{company}/bow-result-index-{index}.xlsx", index=False)

  print(f"Creating {company} bow wordcloud index {index}")
  generateWordCloud(itemList, f"results/{company}/wordcloud-index-{index}.png")

  print(f"Creating {company} index {index} graph file")
  generateHorizontalLineGraph(
    df, 
    f"results/{company}/bow-graph-index-{index}.png", 
    f"BoW {company} index {index}", 
    "Frequency", 
    "Word",
    width = 10 + (index * 4),
    height = 6, 
    sortby = "Frequency",
    ascending = False
  )

# Output # 3 -  N-GRAM 
###### Entrada: Lista de reviews por empresa: `reviewsListPTBR`, `reviewsListNubank`, `reviewsListBB`, `reviewsListItau`
###### Saída: Lista de BoW de 1, 2, 3, 4 e 5 index por empresa

In [ ]:
testedf = pd.DataFrame(bowPayload)
testedf.head(20)

In [ ]:
# Transform list of objetc and list to list of object

newList = []

def wordsToObject(words, company, index):
  for word in words:
    store = company.split('-')[1]
    companyName = company.split('-')[0]
    newList.append({
      'id': index,
      'company': companyName,
      'store': store,
      'word': word
    })

def enumerateList(obj):
  currentCompany = obj['company']
  currentReviews = obj['reviews']
  pprint.pprint(f"Company: {currentCompany}")

  for index, review in enumerate(currentReviews):
    texts = removePunctuation([review])
    words = tokenize(texts)
    words = removeStopwords(words)
    pairs = createPairs(words, 1)
    wordsToObject(pairs, currentCompany, index)

  return newList

def transformListToObjectList(obj):
  for item in obj:
    enumerateList(item)

transformListToObjectList(bowPayload)
  
df = pd.DataFrame(newList)
df.to_csv(f"results/total-words-{reviewsVolume}.csv", index=False)
df.head()

In [ ]:
# Transform list of objetc and list to list of object

newList = []

def wordsToObject(words, company, index):
  for word in words:
    store = company.split('-')[1]
    companyName = company.split('-')[0]
    newList.append({
      'id': index,
      'company': companyName,
      'store': store,
      'word': word
    })

def enumerateList(obj):
  currentCompany = obj['company']
  currentReviews = obj['reviews']
  pprint.pprint(f"Company: {currentCompany}")

  for index, review in enumerate(currentReviews):
    texts = removePunctuation([review])
    words = tokenize(texts)
    words = removeStopwords(words)
    pairs = createPairs(words, 1)
    wordsToObject(pairs, currentCompany, index)

  return newList

def transformListToObjectList(obj):
  for item in obj:
    enumerateList(item)

transformListToObjectList(bowPayload)
  
df = pd.DataFrame(newList)
df.to_csv(f"results/total-words-{reviewsVolume}.csv", index=False)
df.head()

# Output # 4 - TF-IDF em diferentes indexs por bancos
###### Entrada: Lista de reviews por empresa: `reviewsListPTBR`, `reviewsListNubank`, `reviewsListBB`, `reviewsListItau`
###### Saída: Lista de TF-IDF de 1, 2, 3, 4 e 5 index por empresa

In [ ]:
listOfIndex = [1,2,3,4,5]

def calculate_tfidf(texts, index):
    
    textsCleaned = removePunctuation(texts)
    textsWithoutStopwords = removeStopwordsList(textsCleaned)
    
    # Calcular o TF-IDF dos termos
    vectorizer = TfidfVectorizer(ngram_range=(index, index))
    tfidf_matrix = vectorizer.fit_transform(textsWithoutStopwords)

    # Obter os termos e seus respectivos valores de TF-IDF
    feature_names = vectorizer.get_feature_names_out()
    tfidf_values = tfidf_matrix.toarray()[0]

    tfidf_dict = {term: tfidf for term, tfidf in zip(feature_names, tfidf_values)}
    tfidf_df = pd.DataFrame(tfidf_dict.items(), columns=['Term', 'TF-IDF'])
    tfidf_df = tfidf_df.sort_values(by='TF-IDF', ascending=False)
    tfidf_df = tfidf_df[tfidf_df['TF-IDF'] > 0]

    return tfidf_df

def iterateListOfIndex(texts, company):
    
    for index in listOfIndex:
        print(f"Getting {company} index {index} results")
        itemTFIDF = calculate_tfidf(texts, index)
        print(f"Creating {company} index {index} excel file")
        itemTFIDF.to_excel(f"results/{company}/tfidf-index-{index}.xlsx", index=False)
        print(f"Creating {company} index {index} wordcloud file")
        itemList = itemTFIDF.values.tolist()
        generateWordCloud(itemList, f"results/{company}/tfidf-wordcloud-index-{index}.png")
        print(f"Creating {company} index {index} graph file")
        generateHorizontalLineGraph(
            itemTFIDF, 
            f"results/{company}/tfidf-graph-index-{index}.png", 
            f"TF-IDF {company} index {index}", 
            "TF-IDF", 
            "Term",
            width = 10 + (index * 4),
            height = 6,
            sortby = "TF-IDF",
            ascending = False
        )

for item in bowPayload:
    texts = item['reviews'] 
    company = item['company']
    iterateListOfIndex(texts, company)